In [1]:
import pandas as pd
import numpy as np
import statsmodels as sm
from scipy import stats as st
from bioinfokit.analys import stat
import matplotlib.pyplot as plt
import hcp_utils as hcp
from nilearn import plotting as nlp
from nilearn.plotting.cm import _cmap_d as cm
from tabulate import tabulate

C:\Users\sunyi\anaconda3\lib\site-packages\nilearn\datasets\__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
subjects = [100610, 102311, 102816, 104416, 105923,
            108323, 109123, 111514, 114823, 115017,
            115825, 116726, 118225, 125525, 126426,
            128935, 130114, 130518, 131217, 131722,
            132118, 134627, 134829, 135124, 137128,
            140117, 144226, 145834, 146129, 146432,
            146735, 146937, 148133, 150423, 155938,
            156334, 157336, 158035, 158136, 159239,
            162935, 164131, 164636, 165436, 167036,
            167440, 169040, 169343, 169444, 169747]
mse_sub = []
for sub in subjects:
    loc = 'ae_MSE/' + str(sub) + '_regional_MSE_20.csv'
    df = pd.read_csv(loc,index_col=0)
    mse_sub.append(df)    
print('There are ' + str(len(mse_sub)) + ' subjects.')
print('For each subject, there are ' + str(mse_sub[0].shape[0]) + ' sessions.')
print('For each session, there are ' + str(mse_sub[0].shape[1]) + ' regional MSE.')

There are 50 subjects.
For each subject, there are 14 sessions.
For each session, there are 379 regional MSE.


In [3]:
m1 = pd.DataFrame()
for sub in mse_sub:
    m1 = pd.concat([m1, sub.iloc[[0]]], ignore_index = True, axis = 0)
print(m1.shape)
m1.head()

(50, 379)


,L_V1,L_MST,L_V6,L_V2,L_V3,L_V4,L_V8,L_4,L_3b,L_FEF,...,diencephalon_right,hippocampus_left,hippocampus_right,pallidum_left,pallidum_right,putamen_left,putamen_right,thalamus_left,thalamus_right,brainStem
0,0.011017,0.004830,0.010033,0.002520,0.002549,0.002499,0.004675,0.001577,0.003714,0.003422,...,0.005843,0.003384,0.003437,0.017844,0.019864,0.005246,0.007848,0.001474,0.001410,0.005820
1,0.010915,0.006951,0.011236,0.002412,0.003019,0.004317,0.006295,0.002570,0.003489,0.003482,...,0.002619,0.002353,0.002499,0.007286,0.008486,0.003764,0.004628,0.003429,0.003717,0.002505
2,0.008636,0.004741,0.008820,0.002192,0.002600,0.002820,0.005059,0.001508,0.002225,0.004964,...,0.005548,0.003290,0.003287,0.016476,0.018817,0.004258,0.007301,0.002306,0.002381,0.005332
3,0.006112,0.002899,0.011651,0.001647,0.001740,0.003381,0.005640,0.001675,0.002183,0.004224,...,0.000988,0.001931,0.001919,0.001934,0.001629,0.006360,0.007483,0.001929,0.001684,0.000563
4,0.007980,0.005478,0.005818,0.002344,0.002215,0.002553,0.002267,0.000981,0.002243,0.003654,...,0.004645,0.002962,0.002975,0.013859,0.016132,0.004058,0.006278,0.001565,0.001426,0.004586


In [4]:
m_all = pd.DataFrame()
for sub in mse_sub:
    m_all = pd.concat([m_all, sub.iloc[[0]]], ignore_index = True, axis = 0)
    m_all = pd.concat([m_all, sub.iloc[[1]]], ignore_index = True, axis = 0)
    m_all = pd.concat([m_all, sub.iloc[[2]]], ignore_index = True, axis = 0)
    m_all = pd.concat([m_all, sub.iloc[[3]]], ignore_index = True, axis = 0)
    
print(m_all.shape)
m_all.head()

(200, 379)


,L_V1,L_MST,L_V6,L_V2,L_V3,L_V4,L_V8,L_4,L_3b,L_FEF,...,diencephalon_right,hippocampus_left,hippocampus_right,pallidum_left,pallidum_right,putamen_left,putamen_right,thalamus_left,thalamus_right,brainStem
0,0.011017,0.004830,0.010033,0.002520,0.002549,0.002499,0.004675,0.001577,0.003714,0.003422,...,0.005843,0.003384,0.003437,0.017844,0.019864,0.005246,0.007848,0.001474,0.001410,0.005820
1,0.008748,0.006574,0.007975,0.002636,0.002412,0.002034,0.005404,0.002158,0.003033,0.003041,...,0.004493,0.002857,0.002597,0.013452,0.015838,0.004227,0.005770,0.001527,0.001977,0.004744
2,0.009799,0.009289,0.010636,0.002339,0.002519,0.003128,0.006469,0.002617,0.004910,0.002943,...,0.003305,0.002180,0.003006,0.010232,0.011078,0.004280,0.005866,0.002262,0.002055,0.003329
3,0.007503,0.004619,0.007421,0.002797,0.002877,0.002225,0.005164,0.001651,0.004299,0.002739,...,0.004377,0.002289,0.002413,0.012134,0.013734,0.003404,0.004969,0.002021,0.001885,0.004006
4,0.010915,0.006951,0.011236,0.002412,0.003019,0.004317,0.006295,0.002570,0.003489,0.003482,...,0.002619,0.002353,0.002499,0.007286,0.008486,0.003764,0.004628,0.003429,0.003717,0.002505


In [5]:
r1 = pd.DataFrame()
for sub in mse_sub:
    r1 = pd.concat([r1, sub.iloc[[4]]], ignore_index = True, axis = 0)
print(r1.shape)
r1.head()

(50, 379)


,L_V1,L_MST,L_V6,L_V2,L_V3,L_V4,L_V8,L_4,L_3b,L_FEF,...,diencephalon_right,hippocampus_left,hippocampus_right,pallidum_left,pallidum_right,putamen_left,putamen_right,thalamus_left,thalamus_right,brainStem
0,0.006070,0.005455,0.005250,0.002382,0.002577,0.002186,0.004006,0.001900,0.003785,0.003163,...,0.004854,0.003145,0.003182,0.015413,0.017331,0.004585,0.006064,0.001673,0.001499,0.004793
1,0.006188,0.003310,0.003793,0.002258,0.003516,0.003919,0.004323,0.002253,0.002851,0.001227,...,0.001920,0.003557,0.004406,0.005665,0.006650,0.002468,0.002831,0.001438,0.001965,0.001943
2,0.004108,0.004657,0.002764,0.001735,0.001811,0.002473,0.002529,0.001510,0.002261,0.008315,...,0.005528,0.003010,0.002838,0.017593,0.019293,0.004208,0.005730,0.001898,0.001617,0.005815
3,0.005676,0.003786,0.006028,0.002026,0.002414,0.003041,0.005410,0.001660,0.003041,0.003654,...,0.002544,0.002567,0.002420,0.005929,0.005910,0.005738,0.006362,0.001545,0.001878,0.001803
4,0.004991,0.003926,0.002464,0.002091,0.002000,0.002040,0.002699,0.001405,0.001899,0.002462,...,0.003772,0.002650,0.002576,0.011614,0.013214,0.002972,0.004134,0.003796,0.003786,0.004196


In [7]:
r_all = pd.DataFrame()
for sub in mse_sub:
    r_all = pd.concat([r_all, sub.iloc[[4]]], ignore_index = True, axis = 0)
    r_all = pd.concat([r_all, sub.iloc[[5]]], ignore_index = True, axis = 0)
    r_all = pd.concat([r_all, sub.iloc[[6]]], ignore_index = True, axis = 0)
    r_all = pd.concat([r_all, sub.iloc[[7]]], ignore_index = True, axis = 0)
    
print(r_all.shape)
r_all.tail()

(200, 379)


,L_V1,L_MST,L_V6,L_V2,L_V3,L_V4,L_V8,L_4,L_3b,L_FEF,...,diencephalon_right,hippocampus_left,hippocampus_right,pallidum_left,pallidum_right,putamen_left,putamen_right,thalamus_left,thalamus_right,brainStem
195,0.005655,0.002964,0.005148,0.003564,0.004197,0.004366,0.006464,0.004740,0.002349,0.003084,...,0.001643,0.001553,0.001829,0.003278,0.002791,0.002478,0.002105,0.002641,0.003492,0.001309
196,0.004323,0.002274,0.003181,0.002375,0.002740,0.002799,0.006093,0.001572,0.001745,0.002899,...,0.002821,0.001716,0.001978,0.004815,0.003975,0.005587,0.006220,0.003986,0.003700,0.001384
197,0.004535,0.002147,0.001921,0.001925,0.002117,0.002480,0.003704,0.001590,0.001477,0.003763,...,0.001759,0.002019,0.003075,0.003782,0.003123,0.007727,0.008784,0.002913,0.003347,0.001135
198,0.004465,0.002075,0.002377,0.002306,0.002605,0.002218,0.004012,0.001516,0.001906,0.002966,...,0.002687,0.001628,0.001780,0.005526,0.004244,0.007956,0.007842,0.003098,0.003664,0.001752
199,0.004460,0.002090,0.005130,0.001738,0.001582,0.002073,0.003345,0.001596,0.001856,0.004910,...,0.002656,0.001622,0.002246,0.006500,0.005669,0.006028,0.006544,0.002541,0.003052,0.002083


In [8]:
t1 = pd.DataFrame()
for sub in mse_sub:
    t1 = pd.concat([t1, sub.iloc[[8]]], ignore_index = True, axis = 0)
print(t1.shape)
t1.tail()

(50, 379)


,L_V1,L_MST,L_V6,L_V2,L_V3,L_V4,L_V8,L_4,L_3b,L_FEF,...,diencephalon_right,hippocampus_left,hippocampus_right,pallidum_left,pallidum_right,putamen_left,putamen_right,thalamus_left,thalamus_right,brainStem
45,0.003335,0.006173,0.005549,0.002660,0.002961,0.007019,0.009692,0.002894,0.002703,0.004402,...,0.000687,0.002182,0.002219,0.001195,0.001439,0.003972,0.004355,0.001696,0.001753,0.000945
46,0.004868,0.005321,0.006208,0.002947,0.003136,0.006702,0.008494,0.002974,0.004076,0.005998,...,0.003983,0.003369,0.003128,0.010315,0.007664,0.003851,0.003956,0.002463,0.002851,0.002532
47,0.004391,0.004570,0.007663,0.002300,0.002661,0.002067,0.005181,0.001544,0.003536,0.003322,...,0.003122,0.001298,0.002417,0.007951,0.005321,0.001894,0.001737,0.001101,0.001856,0.001997
48,0.002567,0.005897,0.002920,0.001373,0.002272,0.003124,0.010442,0.002226,0.001642,0.002346,...,0.002300,0.000906,0.000913,0.006657,0.006819,0.001608,0.002469,0.003234,0.004832,0.001098
49,0.004330,0.006680,0.007997,0.003881,0.004290,0.006880,0.009349,0.002189,0.002710,0.003505,...,0.002948,0.002734,0.002351,0.005749,0.004631,0.009059,0.010907,0.002838,0.003436,0.001813


# Paired T-Test


In [9]:
t_mr, p_mr_r = st.ttest_rel(m1, r1, alternative='greater')
tf_mr, p_mr = sm.stats.multitest.fdrcorrection(p_mr_r)
count = 0
for n in tf_mr:
    if n:
        count += 1
print("There are " + str(count) + " regions that are significantly different between movie1 and rest1.")
# calculate z values
z_mr = st.norm.ppf(1-p_mr)
z_mr[0:6]

There are 41 regions that are significantly different between movie1 and rest1.


array([ 2.49856084, -1.70436554,  4.02420225,  1.34287466,  0.47157718,
        0.52720794])

In [10]:
regions = m1.columns
for n in range(len(regions)):
    if z_mr[n] < 1.65:
        z_mr[n] = 0
#     if z_rr[n] < 1.65:
#         z_rr[n] = 0
nlp.view_surf(hcp.mesh.inflated,hcp.cortex_data(hcp.unparcellate(z_mr, hcp.mmp)), bg_map=hcp.mesh.sulc, cmap=cm['black_red'], threshold=1.65, symmetric_cmap = False, vmax = 6, vmin = 1.65).save_as_html('z_mr_inf.html')
nlp.view_surf(hcp.mesh.midthickness,hcp.cortex_data(hcp.unparcellate(z_mr, hcp.mmp)), bg_map=hcp.mesh.sulc, cmap=cm['black_red'], threshold=1.65, symmetric_cmap = False, vmax = 6, vmin = 1.65).save_as_html('z_mr_mid.html')

tb = [['Region Name', 'P-Value']]
for n in range(len(regions)):
    p = p_mr[n]
    if p <= 0.05:
        tb.append([regions[n],p])

print(len(tb))
print(tabulate(tb, headers='firstrow', tablefmt='fancy_grid'))

C:\Users\sunyi\anaconda3\lib\site-packages\nilearn\plotting\js_plotting_utils.py:85: UserWarning: choosing both vmin and a threshold is not allowed; setting vmin to 0
  warnings.warn('choosing both vmin and a threshold is not allowed; '


42
╒═══════════════╤═════════════╕
│ Region Name   │     P-Value │
╞═══════════════╪═════════════╡
│ L_V1          │ 0.00623494  │
├───────────────┼─────────────┤
│ L_V6          │ 2.85844e-05 │
├───────────────┼─────────────┤
│ L_4           │ 0.0102753   │
├───────────────┼─────────────┤
│ L_3b          │ 0.0356148   │
├───────────────┼─────────────┤
│ L_PEF         │ 0.0284663   │
├───────────────┼─────────────┤
│ L_V3A         │ 0.00185439  │
├───────────────┼─────────────┤
│ L_V7          │ 0.00393662  │
├───────────────┼─────────────┤
│ L_FFC         │ 0.000816038 │
├───────────────┼─────────────┤
│ L_PIT         │ 0.0164202   │
├───────────────┼─────────────┤
│ L_A1          │ 0.000275548 │
├───────────────┼─────────────┤
│ L_7m          │ 0.0152981   │
├───────────────┼─────────────┤
│ L_1           │ 0.00609871  │
├───────────────┼─────────────┤
│ L_3a          │ 0.000531908 │
├───────────────┼─────────────┤
│ L_43          │ 0.0490697   │
├───────────────┼─────────────┤
│ L_A

In [8]:
t_rr, p_rr_r = st.ttest_rel(t1, r1, alternative='greater')
tf_rr, p_rr = sm.stats.multitest.fdrcorrection(p_rr_r)
count = 0
for n in tf_rr:
    if n:
        count += 1
print("There are " + str(count) + " regions that are significantly different between movie1 and rest1.")
# calculate z values
z_rr = st.norm.ppf(1-p_rr)
z_rr[0:6]

There are 303 regions that are significantly different between movie1 and rest1.


array([-1.27059972,  5.80165672,  4.80818023,  5.01180815,  4.80818023,
        6.80734543])

In [9]:
regions = t1.columns
for n in range(len(regions)):
    if z_rr[n] < 1.65:
        z_rr[n] = 0
nlp.view_surf(hcp.mesh.inflated,hcp.cortex_data(hcp.unparcellate(z_rr, hcp.mmp)), bg_map=hcp.mesh.sulc, cmap=cm['black_red'], threshold=1.65, symmetric_cmap = False, vmax = 6, vmin = 1.65).save_as_html('z_rr.html')

tb = [['Region Name', 'P-Value']]
for n in range(len(regions)):
    p = p_rr[n]
    if p <= 0.05:
        tb.append([regions[n],p])

print(len(tb))
print(tabulate(tb, headers='firstrow', tablefmt='fancy_grid'))

304
╒══════════════════╤═════════════╕
│ Region Name      │     P-Value │
╞══════════════════╪═════════════╡
│ L_MST            │ 3.28314e-09 │
├──────────────────┼─────────────┤
│ L_V6             │ 7.61552e-07 │
├──────────────────┼─────────────┤
│ L_V2             │ 2.69605e-07 │
├──────────────────┼─────────────┤
│ L_V3             │ 7.61552e-07 │
├──────────────────┼─────────────┤
│ L_V4             │ 4.97082e-12 │
├──────────────────┼─────────────┤
│ L_V8             │ 9.28851e-13 │
├──────────────────┼─────────────┤
│ L_4              │ 0.0165841   │
├──────────────────┼─────────────┤
│ L_3b             │ 0.000124993 │
├──────────────────┼─────────────┤
│ L_FEF            │ 0.0175054   │
├──────────────────┼─────────────┤
│ L_PEF            │ 0.015016    │
├──────────────────┼─────────────┤
│ L_55b            │ 0.00115595  │
├──────────────────┼─────────────┤
│ L_V3A            │ 3.54643e-13 │
├──────────────────┼─────────────┤
│ L_RSC            │ 0.000281394 │
├───────────────